In [ ]:
!pip install feedparser

In [ ]:
import requests, base64
import json
import pandas as pd
import feedparser
import re

# https://github.com/kurtmckee/feedparser
# https://wiki.python.org/moin/RssLibraries

In [ ]:
#put app_id in url below

def get_mentions():
    
    results = []
    page_limit = 10
    counter = 1
    
    print(counter)
    first_url = 'https://itunes.apple.com/us/rss/customerreviews/id={app_id}/sortby=mostrecent/xml'
    print(first_url)

    feed = feedparser.parse(first_url)

    for i in feed['entries']:  
        results.append(i) 

    print(len(results))

    next_url = feed['feed']['links'][5]['href']
    
    while counter <= page_limit:

        feed = feedparser.parse(next_url)
        
        for i in feed['entries']:  
            results.append(i)
        
        print(next_url)
        next_url = feed['feed']['links'][5]['href']
        
        counter += 1
        print(counter)
        print(len(results))
                
    return results

In [ ]:
mentions_json = get_mentions()

In [ ]:
def clean_apple_store_reviews(json):
    mentions_df = pd.DataFrame(json)
    mentions_df = mentions_df[['updated',
                                 'id',
                                 'title',
                                 'summary',
                                 'im_votesum',
                                 'im_votecount',
                                 'im_rating',
                                 'im_version',
                                 'href',
                                 'author']]
    mentions_df = mentions_df.drop_duplicates()
    
    mentions_df['title'] = [re.sub(r'\n',' ', x) for x in mentions_df['title']]
    mentions_df['title'] = [re.sub(r'\r',' ', x) for x in mentions_df['title']]
    mentions_df['title'] = [re.sub(r'\\',' ', x) for x in mentions_df['title']]
    mentions_df['title'] = [re.sub(r',','\,', x) for x in mentions_df['title']]
    mentions_df['title'] = [re.sub(r'  ',' ', x) for x in mentions_df['title']]
    mentions_df['title'] = [x.strip() for x in mentions_df['title']]

    mentions_df['summary'] = [re.sub(r'\n',' ', x) for x in mentions_df['summary']]
    mentions_df['summary'] = [re.sub(r'\r',' ', x) for x in mentions_df['summary']]
    mentions_df['summary'] = [re.sub(r'\\',' ', x) for x in mentions_df['summary']]
    mentions_df['summary'] = [re.sub(r',','\,', x) for x in mentions_df['summary']]
    mentions_df['summary'] = [re.sub(r'  ',' ', x) for x in mentions_df['summary']]
    mentions_df['summary'] = [x.strip() for x in mentions_df['summary']]
    
    mentions_df['store'] = 'Apple App Store'
    mentions_df['android_device'] = None
    mentions_df['androidOsVersion'] = None
    mentions_df['android_thumbsUpCount'] = None
    mentions_df['android_thumbsDownCount'] = None
#     mentions_df['android_deviceMetadata'] = None
    
    #rename
    mentions_df = mentions_df.rename(columns={
                                'im_votesum':'apple_votesum',
                                'im_votecount':'apple_votecount',
                                'im_version':'app_version',
                                'href':'apple_href',
                                'im_rating':'rating'                                
                               })
    
    #reorder
    mentions_df = mentions_df[['store',
                             'id',
                             'author',
                             'title',
                             'summary',
                             'rating',
                             'app_version',
                             'apple_href',
                             'apple_votesum',
                             'apple_votecount',
                             'android_device',
                             'androidOsVersion',
                             'android_thumbsUpCount',
                             'android_thumbsDownCount',
#                              'android_deviceMetadata',
                             'updated']]

    return mentions_df
    

In [ ]:
final_df = clean_apple_store_reviews(mentions_json)

In [ ]:
final_df.to_csv('results.csv',index=False)